# Set up environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U openai==0.28.1

In [ ]:
from enum import Enum
from typing import Generic, TypeVar
import torch
import json
import pandas as pd
from pathlib import Path
import time
from tqdm import tqdm as show_progress
import io
import openai
import json
from tqdm import tqdm


# Helper functions

In [ ]:
def create_json_file_with_data(output_file_path: str, data):
    assert output_file_path.endswith('.json')
    with open(output_file_path, 'w') as output_file:
        json.dump(data, output_file, indent=2)

In [ ]:
openai.api_key = 'your_key'

In [ ]:
def paraphrase_text(query: str):
    try:
        result = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            n=3, #Specify how many needed
            temperature=0.6,
            messages=[
                {"role": "system", "content": "You are an English Professor"},
                {"role": "user", "content": query},
            ],
            request_timeout=30,  # Set your desired timeout value in seconds
        )
        return [result.choices[i].message.content for i in range(0, 3)]

    except requests.exceptions.Timeout as e:
        print(f"Timeout occurred: {e}")
        return ['ERROR']

    except Exception as e:
        print(f"An error occurred: {e}")
        return ['ERROR']

In [ ]:
prediction = paraphrase_text("Rephrase the sentence: Mr. President, we found a lot of funny memes about our invasion.  So, we're winning the information war?")
for i in range(0,len(prediction)):
  print(i+1,")",prediction[i])

In [ ]:
# Load the original JSON file
with open('path_to_train_file', 'r') as file:
    original_data = json.load(file)

print(len(original_data))

# Generate 'n' paraphrases

In [ ]:
# Create a list to store the new instances with paraphrased text
new_instances = []

# Use tqdm to add a progress bar
for idx, instance in tqdm(enumerate(original_data), total=len((original_data), desc="Processing Instances", unit="instance"):
    time.sleep(1.5)

    # If we fail, try 2 more times
    for _ in range(3):
        try:
            # Get the original text and other fields
            original_text = instance.get('text', '')  # Default to an empty string if 'text' is not present
            labels = instance.get('labels', [])  # Default to an empty list if 'labels' is not present
            instance_id = instance.get('id', None)  # Default to None if 'id' is not present
            link = instance.get('link', None)  # Default to None if 'link' is not present

            # Paraphrase the text
            query = "Rephrase the sentence: "
            paraphrased_texts = paraphrase_text(query + original_text)

            # Create new instances for each paraphrase
            for i, paraphrased_text in enumerate(paraphrased_texts):
                new_instance = {
                    'instance_id': f"{idx}_{i}",  # Add an instance id with a suffix for each paraphrase
                    'text': paraphrased_text,
                    'labels': labels,  # Keep the original labels
                    'id': instance_id,  # Keep the original id, or set to None if not present
                    'link': link,  # Keep the original link, or set to None if not present
                }

                # Add the new instance to the list
                new_instances.append(new_instance)

            break

        except Exception as e:
            print("An error occurred.")
            print(e)
            print("Waiting for 60 secs before trying again")
            time.sleep(60)
            print("Trying again")

    # Storing intermediate results every 10 instances
    if (idx % 10) == 0:
        print("Storing intermediate results")
        create_json_file_with_data("path_to_result.json", new_instances)

# Save the new instances to a new JSON file
create_json_file_with_data("path_to_result.json", new_instances)
print("Finished")
